### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [406]:
reload(bs)
reload(dl)
reload(fit)

<module 'fitter.fitter' from '../fitter\fitter.pyc'>

### Load Data

In [407]:
data_loader = dl.data_loader()

fit_data = data_loader.get_fit_data()
prior = data_loader.get_prior()

bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=1, bs_N=200)
#bootstrapper.plot_parameters(xy_parameters=['a', 'm_pion'], color_parameter='V')
#data_loader.save_plots(bootstrapper)
#bootstrapper.plot_fit()

### Bootstrap

In [408]:
import time
t0 = time.time()
print bootstrapper
t1 = time.time()
print "\nTotal time (s): ", t1 - t0, "\n"

#squared = lambda x : x**2
#bootstrapper.plot_fit(xfcn=squared, xlabel='$m_{\pi}^2$ (MeV)$^2$')
#bootstrapper.plot_fit()

print bootstrapper.fits[0]

data_loader.save_plots(bootstrapper.plot_fit_bar_graph())
#data_loader.save_prior(bootstrapper.create_prior_from_fit())

Making fits...
100% complete                                            Time (s):  3.17500019073
Getting bs fit parameters...
100% complete 

Fitting to order 1 
Fitted/[Experimental] values at physical point:

		F_K / F_pi = 1.115(41) [1.201(46)]

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
     l_ju 0              -0.9 (1.1)       l_sj 0               1.0 (1.0)
          1              -0.60 (50)            1               1.08 (50)
     l_a2 0             -0.019 (77)       l_pi 0              -0.9 (1.9)
          1            -0.009 (500)            1              -0.70 (50)
    l_vol 0               0.40 (56)       l_ss 0              0.516 (87)
          1               0.36 (50)            1               0.51 (50)
   l_5lam 0             0.0001 (13)       l_ru 0               0.88 (68)
          1           0.0005 (5000)            1               0.89 (50)
      l_x 0          

In [434]:
# xy_parameters is an array (eg, ['mpi', 'Fpi'])
def plot_parameters(self, xy_parameters, color_parameter=None,
                    xfcn=None, xlabel=None, yfcn=None, ylabel=None, show_fit=True):
    if xfcn is None:
        xfcn = lambda x : 1 * x
        xlabel = self._fmt_key_as_latex(xy_parameters[0])
    else:
        # Must also specify xlabel
        xlabel = xlabel

    if yfcn is None:
        yfcn = lambda y : 1 * y
        ylabel = self._fmt_key_as_latex(xy_parameters[1])
    else:
        # Must also specify ylabel
        ylabel = ylabel

    plot_data = {}
    myfcn = [xfcn, yfcn]
    for j, parameter in enumerate(xy_parameters):
        if parameter in ['FK/Fpi', 'FK / Fpi']:
            plot_data[j] = {abbr :  myfcn[j](self.fit_data[abbr]['FK'] /self.fit_data[abbr]['Fpi']) for abbr in self.abbrs}
        else:
            plot_data[j] = {abbr :  myfcn[j](self.fit_data[abbr][parameter]) for abbr in self.abbrs}


    if color_parameter is None:
        color_parameter = 'a'

    if color_parameter in ['a']:
        color_data = {abbr : np.repeat(self.fit_data[abbr]['aw0'][0]/self.w0, self.bs_N).ravel() for abbr in self.abbrs}
    elif color_parameter in ['L']:
        color_data = {abbr : np.repeat(gv.mean(self.fit_data[abbr][color_parameter]), self.bs_N).ravel() for abbr in self.abbrs}
    elif color_parameter == 'V':
        color_data = {abbr : gv.mean(self.fit_data[abbr][color_parameter]).ravel() for abbr in self.abbrs}
    else:
        color_data = {abbr : gv.mean(self.fit_data[abbr][color_parameter]).ravel() for abbr in self.abbrs}

    # Color by lattice spacing/length
    cmap = matplotlib.cm.get_cmap('rainbow')
    min_max = lambda x : [np.min(x), np.max(x)]
    minimum, maximum = min_max(np.concatenate([color_data[abbr] for abbr in self.abbrs]))
    norm = matplotlib.colors.Normalize(vmin=minimum, vmax=maximum)

    # Get scatter plot & color data
    x = np.zeros(self.bs_N * len(self.abbrs))
    y = np.zeros(self.bs_N * len(self.abbrs))
    z = np.zeros(self.bs_N * len(self.abbrs))
    for j, abbr in enumerate(self.abbrs):
        x[j*self.bs_N:(j+1)*self.bs_N] = gv.mean(plot_data[0][abbr])
        y[j*self.bs_N:(j+1)*self.bs_N] = gv.mean(plot_data[1][abbr])
        z[j*self.bs_N:(j+1)*self.bs_N] = color_data[abbr]

    sc = plt.scatter(x, y, c=z, vmin=minimum, vmax=maximum,
                     cmap=cmap, rasterized=True, marker=".", alpha=100.0/self.bs_N)

    if (show_fit) and (xy_parameters[1] in ['FK/Fpi', 'FK / Fpi']):
        hbar_c = 197.327# used to convert to phys units
        
        #print xfcn(self.get_phys_point_data(xy_parameters[0])/hbar_c)
        #plt.axvline(gv.mean(xfcn(self.get_phys_point_data(xy_parameters[0])/hbar_c)))
        
        colors = ['purple', 'green', 'red']
        for j, a in enumerate([0.09, 0.12, 0.15]):
            minimum = np.nanmin([np.nanmin(self.fit_data[abbr][xy_parameters[0]]) for abbr in self.fit_data.keys()])
            maximum = np.nanmax([np.nanmax(self.fit_data[abbr][xy_parameters[0]]) for abbr in self.fit_data.keys()])
            delta = maximum - minimum

            #x = np.linspace(np.max((minimum - 0.05*delta, 0)), maximum + 0.05*delta)
            x = np.linspace(0.0001, maximum + 0.05*delta)

            
            prepped_data = self.get_phys_point_data()
            prepped_data[xy_parameters[0]] = x * hbar_c / a
            prepped_data['aw0'] = a*self.w0

            y = self.fk_fpi_fit_fcn(fit_data=prepped_data)

            pm = lambda g, k : gv.mean(g) + k*gv.sdev(g)
            plt.plot(xfcn(x), pm(y, 0), '--', color=colors[j], label='$a=$%s (fm)'%(str(a)))
            plt.fill_between(xfcn(x), pm(y, -1), pm(y, 1), alpha=0.20, color=colors[j])
            #plt.plot(xfcn(x), pm(y, -1), xfcn(x), pm(y, 1))
            #plt.plot(x, pm(y, 0), '--', color=color, label='$a=$'+a)
            #plt.plot(x, pm(y, -1), x, pm(y, 1), color=color)
            
        plt.legend()
    
    # Plot labels
    plt.grid()
    plt.xlabel(xlabel, fontsize = 24)
    plt.ylabel(ylabel, fontsize = 24)

    # Format colorbar
    color_bar = plt.colorbar(sc)
    color_bar.set_alpha(0.8)
    color_bar.draw_all()
    color_bar.set_label(self._fmt_key_as_latex(color_parameter), fontsize = 24)

    # Set xlim, ylim -- only works if xy_parameters[i] is a vector, not a scalar

    min_max = lambda x : [np.min(x), np.max(x)]
    try:
        xmin, xmax = min_max(np.concatenate([gv.mean(plot_data[0][abbr]) for abbr in self.abbrs]))
        ymin, ymax = min_max(np.concatenate([gv.mean(plot_data[1][abbr]) for abbr in self.abbrs]))
        xdelta = xmax - xmin
        ydelta = ymax - ymin
        plt.xlim(0, xmax+0.05*xdelta) #xmin-0.05*xdelta
        plt.ylim(ymin-0.05*ydelta, ymax+0.05*ydelta)
    except ValueError:
        pass

    fig = plt.gcf()
    plt.close()
    return fig

squared = lambda x : x**2
#plot_parameters(bootstrapper, xy_parameters=['mpi', 'FK/Fpi'], xfcn=squared, xlabel='$m_\pi^2$', color_parameter='a')
#plot_parameters(bootstrapper, xy_parameters=['FK', 'FK/Fpi'], color_parameter='a')